In [ ]:
# mount google drive for engine-related contents
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
# set working directory
import os
# for this to work, please create a folder named 'Chess' inside folder 'Colab Notebooks', or change the directory to your preferences
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/Chess')

# get dependency
!pip install chess

In [ ]:
# game handler block

# import dependencies
import chess
import time
from IPython.display import display, HTML, clear_output

# util function
def to_uci(uci=''):
  return chess.Move.from_uci(uci).uci()
def display_board(board, use_svg):
  if use_svg:
    return board._repr_svg_()
  else:
    return "<pre>" + str(board) + "</pre>"

# game class
class Game:
  def __init__(self) -> None:
    self.board = chess.Board()
    self.maxMove = 100 # number of max moves allow in one game
    self.gameState = True
    
  # this is for the final evaluation of the board
  def staticAnalysis(self, color):
    score = 0
    for (piece, value) in [(chess.PAWN, 1), 
                            (chess.BISHOP, 4), 
                            (chess.KING, 0), 
                            (chess.QUEEN, 10), 
                            (chess.KNIGHT, 5),
                            (chess.ROOK, 3)]:
      score += len(self.board.pieces(piece, color)) * value
    return score

  # return the state of the board, playing or not
  def getState(self):
    return self.gameState

  # return current board's FEN position
  def getFen(self):
    return self.board.fen()

  # return WHITE/ BLACK turn
  def getTurn(self):
    return self.board.turn

  # update/ make a move on the board
  def updateBoard(self, uci, time_taken):
    t0 = time_taken
    # game ends if one player failed to return a move within time_limit
    time_limit = 3
    if t0 > time_limit:
      wEval = -9999 if self.board.turn == chess.WHITE else 9999
      bEval = -1 * wEval
      self.gameState = False
      return [not self.board.turn, len(self.board.move_stack), wEval, bEval]
    
    # push the uci-formatted move to the chess board
    try:
      uci = to_uci(uci)
    except:
      uci = uci.uci()
    self.board.push_uci(uci)
    
    # display game
    name = 'White' if self.board.turn == chess.WHITE else 'Black'
    board_stop = display_board(self.board, True)    # using svg
    html = html = "<b>Move %s %s, Play '%s', Time %s:</b><br/>%s" % (
                       len(self.board.move_stack), name, uci, str(t0), board_stop)
    clear_output(wait=True)
    display(HTML(html))
    time.sleep(0.5)

    # check for checkmate
    if self.board.is_checkmate():
      wEval = 9999 if self.board.turn == chess.WHITE else -9999
      bEval = -1 * wEval
      self.gameState = False
      return [self.board.turn, len(self.board.move_stack), wEval, bEval]

    # check for number of moves
    if len(self.board.move_stack) >= self.maxMove:
      wEval, bEval = self.staticAnalysis(chess.WHITE), self.staticAnalysis(chess.BLACK)
      self.gameState = False
      return [chess.WHITE if wEval > bEval else chess.BLACK, len(self.board.move_stack), wEval, bEval]

    # notify game handler that's the game haven't end
    return [True]

Description on how to use this notebook:

- Engine MUST have 2 functions: 
  - update self.board using chess.set_fen() 
  - get_move() which return an UCI move

In [ ]:
# code cell for player 1
import random

# this is a demo class
class PlayerOne:
  def __init__(self) -> None:
    # has a board itself to 'find moves' on
    self.board = chess.Board()
  
  # update the board using FEN-position
  def update(self, fen):
    self.board.set_fen(fen)
  
  # return a move 
  def getMove(self, fen):
    self.update(fen)
    return random.choice(list(self.board.legal_moves))

In [ ]:
# code cell for player 2
import random

# this is a demo class
class PlayerTwo:
  def __init__(self) -> None:
    self.board = chess.Board()
  def update(self, fen):
    self.board.set_fen(fen)
  def getMove(self, fen):
    self.update(fen)
    time.sleep(4)
    return random.choice(list(self.board.legal_moves))

In [ ]:
# game cell (logic handler for both players)

# create game_handler and two players
game_handler = Game()
p1 = PlayerOne()
p2 = PlayerTwo()

# while the game haven't end
while game_handler.getState():
  # get the FEN position of the board
  game_fen = game_handler.getFen()
  # get start time
  t0 = time.time()
  # player (or bot) finds and return the move
  move = p1.getMove(game_fen) if game_handler.getTurn() == chess.WHITE else p2.getMove(game_fen)
  # get end time for finding move
  time_taken = time.time() - t0
  # forward the move, and finding time to the game_handler
  result = game_handler.updateBoard(move, time_taken)

  # if returned result indicate game's end, process the results
  if len(result) > 1:
    # FORMAT: [winner side, move_num, whitePoint, blackPoint]
    winner = 'White' if result[0] == chess.WHITE else 'Black'
    move_num = result[1]
    wPoint = result[2]
    bPoint = result[3]
    print(f'Game end! {winner} wins, {str(move_num)} moves taken, white\'s side point: {str(wPoint)}, black\'s side point: {str(bPoint)}')
    break